# Popularity Contests: Python Lists vs Numpy Arrays

## Abstract

abstract goes here

## Introduction

introduction goes here

In [2]:
import subprocess

print("alive")

# list of repositories to download. These are the first n most forked repositories with a few exceptions.\
repository_urls = ('https://github.com/odoo/odoo.git',
                   'https://github.com/django/django.git', 
                   'https://github.com/scikit-learn/scikit-learn.git', 
                   'https://github.com/mitsuhiko/flask.git',
                   'https://github.com/ansible/ansible.git',
                   'https://github.com/scrapy/scrapy.git',
                   'https://github.com/vinta/awesome-python.git',
                   'https://github.com/tornadoweb/tornado.git',
                   'https://github.com/kennethreitz/requests.git',
                   'https://github.com/rg3/youtube-dl.git',
                   'https://github.com/saltstack/salt.git',
                   'https://github.com/wbond/package_control_channel.git',
                   'https://github.com/udacity/fullstack-nanodegree-vm.git',
                   'https://github.com/ipython/ipython.git',
                   'https://github.com/josephmisiti/awesome-machine-learning.git',
                   'https://github.com/pydata/pandas.git',
                   'https://github.com/reddit/reddit.git',
                   'https://github.com/XX-net/XX-Net.git',
                   'https://github.com/midgetspy/Sick-Beard.git')


def download_repo(url):
    print('starting')
    print(subprocess.Popen("git clone " + url, shell=True, stdout=subprocess.PIPE).stdout.read())
    print('done')

download_repo(repository_urls[0])

alive
starting


KeyboardInterrupt: 

Stuff to track:

-imports

-comments per line

-pep checker score

-coppied code from stack overflow
